In [1]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import pandas as pd
import numpy as np
import re
import datetime
from dateutil.relativedelta import relativedelta, WE
from datetime import datetime,timedelta
from pandas.tseries.offsets import DateOffset, BDay
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import pandas_datareader.data as web
import ffn
import pickle
from io import StringIO

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=[20,10]

import holoviews as hv
import hvplot.pandas
hv.extension('bokeh')

In [ ]:
path=r'F:\WQU_capstone'

In [ ]:
## Download VIX Futures data from CBOE after 2013
mainlink='https://markets.cboe.com/us/futures/market_statistics/historical_data/'
url = requests.get(mainlink).text
soup = BeautifulSoup(url)

cols=['Trade Date','Futures','Open','High','Low','Close','Settle','Change','Total Volume','EFP','Open Interest']
df=pd.DataFrame()
for link in soup.findAll("a",class_='historical-data-link'):
    current_link = link.get("href")
    if (current_link.find('/csv/')!=-1)&(current_link.find('VX')!=-1):
        if(link.text.strip().find('VX+VXT ')!=-1)&(int(re.search('[\d]+',link.text.strip())[0])>13):
            print(current_link)
            df= df.append(pd.read_csv(mainlink+current_link)\
                      ,ignore_index=True)

In [ ]:
## Download VIX Futures data from CBOE before 2014

mainlink2='https://www.cboe.com/products/futures/market-data/historical-data-archive'

url = requests.get(mainlink2).text
soup = BeautifulSoup(url)

df1=pd.DataFrame()
for link in soup.findAll("a"):
    current_link2 = link.get("href")
    if(current_link2!=None):
        if (current_link2.endswith('VX.csv')):
            if (re.search('[\d]+',current_link2)!=None):
                if(int((re.search('[\d]+',current_link2)[0]))<14):
                    print(current_link2)
                    df1= df1.append(pd.read_csv('https://www.cboe.com'+current_link2,names=cols,skiprows=1,header=None)\
                      ,ignore_index=True)


In [ ]:
df1=df1[~(df1['Trade Date']=='Trade Date')].copy()
df1['series'] =df1['Futures'].apply(lambda x:x[0])
df1['year']=df1['Futures'].apply(lambda x:int(x[-3:-1])+2000)
df1['month']=df1['Futures'].apply(lambda x:datetime.strptime(x[-7:-4],'%b').month)
df['series'] =df['Futures'].apply(lambda x:x[0])
df['year']=df['Futures'].apply(lambda x:int(x[-6:-1]))
df['month']=df['Futures'].apply(lambda x:datetime.strptime(x[-9:-6],'%b').month)

In [ ]:
newdf=df.append(df1)
# newdf.drop_duplicates(subset=cols, inplace=True) ## does not serve intended purpose
newdf.to_csv(path+'\\all_fut_data.csv')

In [ ]:
dts= pd.read_excel(path+'\\vix_expirationdts.xlsx')
usbd=CustomBusinessDay(calendar=USFederalHolidayCalendar())
dts['last trading dts']= dts['Expiration dts']-usbd

dts['month']= dts['Expiration dts'].dt.month
dts['year']= dts['Expiration dts'].dt.year

In [ ]:
alldf= pd.merge(newdf,dts, left_on=['year','month'],right_on=['year','month'],how='left')

alldf['Trade Date']= pd.to_datetime(alldf['Trade Date'])
alldf['Expiration dts']= pd.to_datetime(alldf['Expiration dts'])
alldf['Settle']= alldf['Settle'].astype(float)

In [ ]:
def CMT_VIX_FUTURES(m,alldf):
    cutoffdate=pd.to_datetime('2006-11-14') ## cut off date for first month futures is 2006-09-19
    last=cutoffdate

    idx= pd.DataFrame (columns=['date','c1','c2','last_exp','lasttrade_dt','c1_wt','c2_wt','c1_px','c2_px','c1_open','cdr','indexlevel','prev_c1_wt','rollcost'])
    idx['date']=alldf[alldf['Trade Date'] >= cutoffdate]['Trade Date'].unique()##earlier was set at 2005-12-20

    idx.sort_values(by='date',inplace=True)

    idx.set_index('date',inplace=True)

    for i in idx.index:
        getfwdcontracts= alldf[alldf['Expiration dts']>i].sort_values(by='Expiration dts')[['Futures','Expiration dts','last trading dts']].drop_duplicates()
        getprevcontracts= alldf[alldf['Expiration dts']<=i].sort_values(by='Expiration dts',ascending=False)[['Futures','Expiration dts','last trading dts']].drop_duplicates()
        idx.loc[i,'c1']= getfwdcontracts.iloc[m-1,getfwdcontracts.columns.get_loc('Futures')]
        idx.loc[i,'c2']= getfwdcontracts.iloc[m,getfwdcontracts.columns.get_loc('Futures')]
        idx.loc[i,'last_exp']=getprevcontracts.iloc[0,getprevcontracts.columns.get_loc('Expiration dts')]
        idx.loc[i,'lasttrade_dt']=getfwdcontracts.iloc[0,getfwdcontracts.columns.get_loc('last trading dts')]

        days_remaining= len(pd.date_range(start=i,end=idx.loc[i,'lasttrade_dt'],freq=usbd))
        total_days= len(pd.date_range(start=idx.loc[i,'last_exp'],end=idx.loc[i,'lasttrade_dt'],freq=usbd))

        idx.loc[i,'c1_wt']= days_remaining/total_days
        idx.loc[i,'c2_wt']= 1-days_remaining/total_days

        try:
            idx.loc[i,'c1_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c1'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c1_px']=np.NaN
        try:
            idx.loc[i,'c2_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c2'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c2_px']=np.NaN

        if(days_remaining==total_days):
            idx.loc[i,'c1_open']= idx.loc[last,'c2_px']
            idx.loc[i,'cdr']=idx.loc[i,'c1_wt']* idx.loc[i,'c1_px']/idx.loc[i,'c1_open']
            
            idx.loc[i,'prev_c1_wt']= idx.loc[last,'c2_wt']

        else:
            idx.loc[i,'c1_open']= idx.loc[last,'c1_px']
            idx.loc[i,'cdr']=idx.loc[i,'c1_wt']* idx.loc[i,'c1_px']/idx.loc[i,'c1_open']+ idx.loc[i,'c2_wt']*idx.loc[i,'c2_px']/idx.loc[last,'c2_px']
            
            idx.loc[i,'prev_c1_wt']= idx.loc[last,'c1_wt']
        
        if(i!=idx.index[0]):
            idx.loc[last,'rollcost']= (idx.loc[last,'c2_px']-idx.loc[last,'c1_px'])/idx.loc[last,'c2_px']*np.abs(idx.loc[i,'c1_wt']-idx.loc[i,'prev_c1_wt'])

        
        if(i==idx.index[0]):
            idx.loc[i,'indexlevel']=100000
        else:
            idx.loc[i,'indexlevel']= idx.loc[last,'indexlevel']*idx.loc[i,'cdr']
        
        last =i
    
    idx= idx.rename(columns={'indexlevel':str(m)+'M_'+'ER','rollcost':'rollcost_'+str(m)+'M'})
        
    return idx

In [ ]:
dfdict={}
for i in range(4):
    dfdict[str(i+1)+'MER']= CMT_VIX_FUTURES(i+1,alldf)

In [ ]:
vix_idxs=pd.concat([dfdict[str(i+1)+'MER'][str(i+1)+'M_'+'ER'] for i in range(4)],axis=1,join='inner')
rollcosts= pd.concat([dfdict[str(i+1)+'MER']['rollcost_'+str(i+1)+'M'] for i in range(4)],axis=1,join='inner')

In [ ]:
vix_idxs[vix_idxs.index>pd.to_datetime('2008-01-01')].hvplot.line(frame_width=700,frame_height=400)

In [ ]:
def CMT_VIX_FUTURES_MT(m,alldf):
    cutoffdate=pd.to_datetime('2008-05-20')##earlier was set at 2005-12-20
    last=cutoffdate

    idx= pd.DataFrame (columns=['date','c1','c2','c3','c4','last_exp','lasttrade_dt','c1_wt','c2_wt','c3_wt','c4_wt'\
                                ,'c1_px','c2_px','c3_px','c4_px','c1_open','c2_open','c3_open'\
                                ,'cdr','indexlevel','prev_c1_wt','rollcost'])
    idx['date']=alldf[alldf['Trade Date'] >= cutoffdate]['Trade Date'].unique()

    idx.sort_values(by='date',inplace=True)
    idx.loc[idx.index[0],'indexlevel']=100000

    idx.set_index('date',inplace=True)

    for i in idx.index:
        getfwdcontracts= alldf[alldf['Expiration dts']>i].sort_values(by='Expiration dts')[['Futures','Expiration dts','last trading dts']].drop_duplicates()
        getprevcontracts= alldf[alldf['Expiration dts']<=i].sort_values(by='Expiration dts',ascending=False)[['Futures','Expiration dts','last trading dts']].drop_duplicates()
        idx.loc[i,'c1']= getfwdcontracts.iloc[m-1,getfwdcontracts.columns.get_loc('Futures')]
        idx.loc[i,'c2']= getfwdcontracts.iloc[m,getfwdcontracts.columns.get_loc('Futures')]
        idx.loc[i,'c3']= getfwdcontracts.iloc[m+1,getfwdcontracts.columns.get_loc('Futures')]
        idx.loc[i,'c4']= getfwdcontracts.iloc[m+2,getfwdcontracts.columns.get_loc('Futures')]
        
        idx.loc[i,'last_exp']=getprevcontracts.iloc[0,getprevcontracts.columns.get_loc('Expiration dts')]
        idx.loc[i,'lasttrade_dt']=getfwdcontracts.iloc[0,getfwdcontracts.columns.get_loc('last trading dts')]

        days_remaining= len(pd.date_range(start=i,end=idx.loc[i,'lasttrade_dt'],freq=usbd))
        total_days= len(pd.date_range(start=idx.loc[i,'last_exp'],end=idx.loc[i,'lasttrade_dt'],freq=usbd))

        idx.loc[i,'c1_wt']= days_remaining/total_days* (1/3)
        idx.loc[i,'c4_wt']= (1/3)*(1-days_remaining/total_days)
        idx.loc[i,'c2_wt']= (1/3)
        idx.loc[i,'c3_wt']= (1/3)

        try:
            idx.loc[i,'c1_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c1'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c1_px']=np.NaN
        try:
            idx.loc[i,'c2_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c2'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c2_px']=np.NaN
            
        try:
            idx.loc[i,'c3_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c3'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c3_px']=np.NaN
            
        try:
            idx.loc[i,'c4_px']= alldf.loc[((alldf.Futures==idx.loc[i,'c4'])&(alldf['Trade Date']==i)),'Settle'].values[0]
        except:
            idx.loc[i,'c4_px']=np.NaN



        if(days_remaining==total_days):
            idx.loc[i,'c1_open']= idx.loc[last,'c2_px']
            idx.loc[i,'c2_open']= idx.loc[last,'c3_px']
            idx.loc[i,'c3_open']= idx.loc[last,'c4_px']
            
            idx.loc[i,'cdr']=idx.loc[i,'c1_wt']* idx.loc[i,'c1_px']/idx.loc[i,'c1_open']+\
            idx.loc[i,'c2_wt']* idx.loc[i,'c2_px']/idx.loc[i,'c2_open']+\
            idx.loc[i,'c3_wt']* idx.loc[i,'c3_px']/idx.loc[i,'c3_open']
            
            idx.loc[i,'prev_c1_wt']= idx.loc[last,'c4_wt']
            
        else:
            idx.loc[i,'c1_open']= idx.loc[last,'c1_px']
            idx.loc[i,'c2_open']= idx.loc[last,'c2_px']
            idx.loc[i,'c3_open']= idx.loc[last,'c3_px']
            
            idx.loc[i,'cdr']= idx.loc[i,'c1_wt']* idx.loc[i,'c1_px']/idx.loc[i,'c1_open']+\
            idx.loc[i,'c2_wt']* idx.loc[i,'c2_px']/idx.loc[i,'c2_open']+\
            idx.loc[i,'c3_wt']* idx.loc[i,'c3_px']/idx.loc[i,'c3_open']+\
            idx.loc[i,'c4_wt']* idx.loc[i,'c4_px']/idx.loc[last,'c4_px']
            
            idx.loc[i,'prev_c1_wt']= idx.loc[last,'c1_wt']
        
        if(i!=idx.index[0]):
            idx.loc[last,'rollcost']= (idx.loc[last,'c4_px']-idx.loc[last,'c1_px'])/idx.loc[last,'c4_px']*np.abs(idx.loc[i,'c1_wt']-idx.loc[i,'prev_c1_wt'])
        
        if(i==idx.index[0]):
            idx.loc[i,'indexlevel']=100000
        else:
            idx.loc[i,'indexlevel']= idx.loc[last,'indexlevel']*idx.loc[i,'cdr']
            
        last =i

    idx= idx.rename(columns={'indexlevel':'4M_MediumTerm'+'ER','rollcost':'rollcost_4MMT'})

    return idx

In [ ]:
dfdict['4M_MTER']=CMT_VIX_FUTURES_MT(4,alldf)

In [ ]:
vix_idxs=vix_idxs.join(dfdict['4M_MTER']['4M_MediumTermER'],how='inner')
rollcosts= rollcosts.join(dfdict['4M_MTER']['rollcost_4MMT'],how='inner')

In [ ]:
alldfcopy= alldf[alldf['Expiration dts'].notna()].copy()
alldfcopy['Trade Date']=alldfcopy['Trade Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

def plotter(date='2006-11-14'):
    snap= alldfcopy[alldfcopy['Trade Date']==date]
    snap.sort_values(by='Expiration dts',inplace=True)
    snap.rename(columns={'Settle':'Settement Price','Futures':'Futures Series'},inplace=True)
    plot = hv.Curve(snap,vdims='Settement Price',kdims='Futures Series').options(color='red',line_width=2)
    return plot

%opts Curve [width=650, height=400]
hv.DynamicMap(plotter, kdims='date', label="Term Structure of VIX Futures").redim.values(date=alldfcopy['Trade Date'].sort_values(ascending=False).unique())

In [ ]:
b=pd.read_excel(r'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixarchive.xls',skiprows=1)
a=pd.read_csv(r'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv',skiprows=1,parse_dates=['Date'])

vix=b.append(a,ignore_index=True).set_index('Date')
vix.rename(columns={'VIX Close':'VIX'},inplace=True)

In [ ]:
from holoviews.operation.timeseries import rolling, rolling_outlier_std
from holoviews import opts

curve= hv.Curve(vix['VIX'])
scatter=hv.Scatter(rolling_outlier_std(curve,rolling_window=7500,min_periods=1250,sigma=2))
spike=hv.Spikes(scatter)

curve*scatter*spike.opts(color='pink',alpha=2,width=800,height=500)

In [ ]:
vix.hvplot.kde(y='VIX',width=700,height=500,title='KDE of VIX Index Levels')

In [ ]:
## Extract data (Closing px adj for splits and dividends) from Yahoo finance
start_dt=pd.to_datetime('1990-01-01')
SPX = web.DataReader('SPY', 'yahoo',start=start_dt).drop(columns=['High','Low', 'Open','Close','Volume']).rename(columns={'Adj Close':'SPY'})

In [ ]:
all_idxs=ffn.core.rebase(pd.concat([vix_idxs,vix['VIX'],SPX['SPY']],join='inner',axis=1),value=100000)

In [ ]:
all_idxs=ffn.core.rebase(vix_idxs.join(vix['VIX'],how='left').join(SPX['SPY'],how='left').fillna(method='ffill'),value=100000)

In [ ]:
all_idxs.hvplot.line(y=[str(i+1)+'M_ER' for i in range(4)]+['4M_MediumTermER','SPY','VIX'],width=1200, height=700)

In [ ]:
((rollcosts/1e-4).dropna(how='all')).hvplot.line(y=['rollcost_1M','rollcost_4MMT'],title='Roll Costs',width=1200, height=700)*hv.HLine(0).opts(color='black',line_dash='dashed',alpha=0.2)

In [ ]:
(rollcosts/1e-4).mean()

In [ ]:
##Pickle all the vix as constant matuurity vix futures index levels, along with all other relevant data
pickle_vix = open(path+"\\VIX_indexes.pkl","wb")
pickle_alldata = open(path+"\\alldata_dict.pkl",'wb')
pickle_rollcosts= open(path+"\\rollcosts.pkl",'wb')
pickle.dump(all_idxs, pickle_vix)
pickle.dump(dfdict,pickle_alldata)
pickle.dump((rollcosts/1e-4).dropna(how='all'),pickle_rollcosts)
pickle_vix.close()
pickle_alldata.close()
pickle_rollcosts.close()

In [ ]:
## VSTOXX data
s2= requests.get('https://www.stoxx.com/document/Indices/Current/HistoricalData/h_v2tx.txt').text
vstoxx=pd.read_csv(StringIO(s2),sep=';')

s=requests.get('https://www.stoxx.com/document/Indices/Current/HistoricalData/h_vst1me.txt').text
vst1me=pd.read_csv(StringIO(s),sep=';',skiprows=1)
s1= requests.get('https://www.stoxx.com/document/Indices/Current/HistoricalData/h_vmt5me.txt').text
vmt5me=pd.read_csv(StringIO(s1),sep=';',skiprows=1)
    
# pd.read_csv('https://www.stoxx.com/document/Indices/Current/HistoricalData/h_vst1me.txt',sep=';',encoding='utf-8')

vst1me['Date']= pd.to_datetime(vst1me['Date'])
vmt5me['Date']= pd.to_datetime(vmt5me['Date'])
vstoxx['Date']= pd.to_datetime(vstoxx['Date'])


STOXX = web.DataReader('^STOXX50E', 'yahoo',start=start_dt).drop(columns=['High','Low', 'Open','Close','Volume']).rename(columns={'Adj Close':'STOXX'})

vstoxx= pd.concat([STOXX,vstoxx.set_index('Date'),vst1me.set_index('Date'),vmt5me.set_index('Date')],join='inner',axis=1)

In [ ]:
pickle_vstoxx= open(path+"\\VSTOXX_indexes.pkl","wb")
pickle.dump(vstoxx, pickle_vstoxx)
pickle_vstoxx.close()